In [118]:
import pandas as pd
import numpy as np
import os
import json
import re
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tflearn
import nltk
from nltk.stem.lancaster import LancasterStemmer
from sklearn.metrics import confusion_matrix

In [98]:
dfload = pd.read_json(os.getcwd()+'/../Data/convertcsv.json')
df = pd.DataFrame(columns=['TAGS','PATTERNS','RESPONSES'])
i = 0
for _,r in dfload.iterrows():
    for resp in r[0]['response']:
        for p in r[0]['patterns']:
            df.set_value(i, 'PATTERNS', p)
            df.set_value(i, 'TAGS', r[0]['tag'])
            df.set_value(i, 'RESPONSES', resp)
            i += 1
df

F:\anaconda\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
F:\anaconda\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  
F:\anaconda\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


,TAGS,PATTERNS,RESPONSES
0,greeting,Hi,"Hey there, friend! I'm InvestoBot, the chatbot..."
1,greeting,Hey,"Hey there, friend! I'm InvestoBot, the chatbot..."
2,greeting,Hello,"Hey there, friend! I'm InvestoBot, the chatbot..."
3,greeting,I am fine. And you?,"Hey there, friend! I'm InvestoBot, the chatbot..."
4,greeting,How are you,"Hey there, friend! I'm InvestoBot, the chatbot..."
5,greeting,Is there anyone,"Hey there, friend! I'm InvestoBot, the chatbot..."
6,greeting,"Good Morning, I need some help","Hey there, friend! I'm InvestoBot, the chatbot..."
7,greeting,Howdy !!,"Hey there, friend! I'm InvestoBot, the chatbot..."
8,greeting,"Hey, I have a question regarding investments.","Hey there, friend! I'm InvestoBot, the chatbot..."
9,greeting,"Hello, Good Day ! How are you ?","Hey there, friend! I'm InvestoBot, the chatbot..."


In [99]:
# df = pd.read_csv(os.getcwd()+'/../Data/NLPData.csv')
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [100]:
class PreProcessMessage():

    def __init__(self):
        pass

    def preprocess_word(self, word):
        # Remove punctuation
        word = word.strip('\'"?!,.():;')
        # Convert more than 2 letter repetitions to 2 letter
        # funnnnny --> funny
        word = re.sub(r'(.)\1+', r'\1\1', word)
        # Remove - & '
        word = re.sub(r'(-|\')', '', word)
        return word


    def is_valid_word(self, word):
        # Check if word begins with an alphabet
        return (re.search(r'^[a-zA-Z][a-z0-9A-Z\._]*$', word) is not None)


    def handle_emojis(self, message):
        # Smile -- :), : ), :-), (:, ( :, (-:, :')
        message = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', message)
        # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
        message = re.sub(r'(:\s?d|:-d|x-?d|x-?d)', ' EMO_POS ', message)
        # Love -- <3, :*
        message = re.sub(r'(<3|:\*)', ' EMO_POS ', message)
        # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
        message = re.sub(r'(;-?\)|;-?d|\(-?;)', ' EMO_POS ', message)
        # Sad -- :-(, : (, :(, ):, )-:
        message = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', message)
        # Cry -- :,(, :'(, :"(
        message = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', message)
        return message

    def preprocess_message(self, message):
        processed_message = []
        # Convert to lower case
        message = message.lower()
        # Replaces URLs with the word URL
        message = re.sub(r'((www\.[\S]+)|(https?://[\S]+))', ' URL ', message)
        # Replace 2+ dots with space
        message = re.sub(r'\.{2,}', ' ', message)
        # Strip space, " and ' from message
        message = message.strip(' "\'')
        # Replace emojis with either EMO_POS or EMO_NEG
        message = self.handle_emojis(message)
        # Replace multiple spaces with a single space
        message = re.sub(r'\s+', ' ', message)
        words = message.split()

        for word in words:
            word = self.preprocess_word(word)
            if self.is_valid_word(word):
                processed_message.append(word)
        return ' '.join(processed_message)

    def transformCleanedMessage(self, message):
        tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
        transformedMessage=tfv.fit_transform(message)
        return transformedMessage

In [108]:
newdf = df
print(newdf.TAGS.nunique())
print(newdf.TAGS.unique())

20
[greeting, name, yesAccount, noAccount, accountNumberEntered, ..., mutualFundsInvestment, stocksInvestment, yesStockInvestment, noStockInvestment, exit]
Length: 20
Categories (20, object): [greeting, name, yesAccount, noAccount, ..., stocksInvestment, yesStockInvestment, noStockInvestment, exit]


In [109]:
newdf.TAGS = pd.Categorical(newdf.TAGS)
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 3 columns):
TAGS         147 non-null category
PATTERNS     147 non-null object
RESPONSES    147 non-null object
dtypes: category(1), object(2)
memory usage: 3.3+ KB


In [110]:
le = LabelEncoder()
newdf['Target'] = le.fit_transform(newdf.TAGS)
newdf

,TAGS,PATTERNS,RESPONSES,Target
0,greeting,Hi,"Hey there, friend! I'm InvestoBot, the chatbot...",6
1,greeting,Hey,"Hey there, friend! I'm InvestoBot, the chatbot...",6
2,greeting,Hello,"Hey there, friend! I'm InvestoBot, the chatbot...",6
3,greeting,I am fine. And you?,"Hey there, friend! I'm InvestoBot, the chatbot...",6
4,greeting,How are you,"Hey there, friend! I'm InvestoBot, the chatbot...",6
5,greeting,Is there anyone,"Hey there, friend! I'm InvestoBot, the chatbot...",6
6,greeting,"Good Morning, I need some help","Hey there, friend! I'm InvestoBot, the chatbot...",6
7,greeting,Howdy !!,"Hey there, friend! I'm InvestoBot, the chatbot...",6
8,greeting,"Hey, I have a question regarding investments.","Hey there, friend! I'm InvestoBot, the chatbot...",6
9,greeting,"Hello, Good Day ! How are you ?","Hey there, friend! I'm InvestoBot, the chatbot...",6


In [111]:
processed_set = []
only_text = pd.DataFrame(newdf['PATTERNS'])
for i in range(0, len(newdf)):
    processed = PreProcessMessage().preprocess_message(only_text['PATTERNS'][i])
    processed_set.append(processed)

In [112]:
X_train, X_test, y_train, y_test = train_test_split(processed_set,newdf.Target, test_size=0.2, random_state=42)

In [113]:
tfv=TfidfVectorizer(min_df=0, max_features=None, strip_accents='unicode',lowercase =True,
        analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1), sublinear_tf=True, stop_words = "english")
X_train_transformed=tfv.fit_transform(X_train)
X_test_transformed=tfv.transform(X_test)
processed_set_transformed = tfv.fit_transform(processed_set)

In [114]:
processed_set_transformed.shape

(147, 75)

In [126]:
lr=LogisticRegression(multi_class='ovr')
lr.fit(processed_set_transformed,newdf.Target)
lr.score(processed_set_transformed, newdf.Target)

F:\anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7687074829931972

In [130]:
def printModelEvaluationMetrics(true, pred, cols):
    cnf_matrix = confusion_matrix(true, pred)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)

    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
#     print('Sensitivity, hit rate, recall, or true positive rate is ' + str(TPR))
    # Specificity or true negative rate
    TNR = TN/(TN+FP) 
#     print('Specificity or true negative rate is ' + str(TNR))
    # Precision or positive predictive value
    PPV = TP/(TP+FP)
#     print('Precision or positive predictive value is ' + str(PPV))
    # Negative predictive value
    NPV = TN/(TN+FN)
#     print('Negative predictive value is ' + str(NPV))
    # Fall out or false positive rate
    FPR = FP/(FP+TN)
#     print('Fall out or false positive rate is ' + str(FPR))
    # False negative rate
    FNR = FN/(TP+FN)
#     print('False negative rate is ' + str(FNR))
    # False discovery rate
    FDR = FP/(TP+FP)
#     print('False discovery rate is ' + str(FDR))
    # Overall accuracy
    ACC = (TP+TN)/(TP+FP+FN+TN)
#     print('Overall accuracy is ' + str(ACC))
    ind = ['Sensitivity/Recall/TPR','Specificity/TNR','Precision/PPV','NegativePredictiveValue','FallOut/FPR',
          'FalseNegativeRate','FalseDiscoveryRate','Accuracy']
    data = [TPR, TNR, PPV, NPV, FPR, FNR, FDR, ACC]
    evaldf = pd.DataFrame(columns=cols, data=data, index=ind)
    return evaldf

In [134]:
eval = printModelEvaluationMetrics(np.array(newdf.Target), np.array(lr.predict(processed_set_transformed)), le.inverse_transform(newdf.Target.unique()))
eval.to_csv(os.getcwd() + '/../evaluation/logistic.csv')

F:\anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
F:\anaconda\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


## XGBoost

In [135]:
from xgboost import XGBClassifier
xgb = XGBClassifier(objective='multi:softmax', num_class=newdf.Target.nunique)
xgb.fit(processed_set_transformed,newdf.Target)
xgb.score(processed_set_transformed, newdf.Target)

0.8503401360544217

In [136]:
eval = printModelEvaluationMetrics(np.array(newdf.Target), np.array(xgb.predict(processed_set_transformed)), le.inverse_transform(newdf.Target.unique()))
eval.to_csv(os.getcwd() + '/../evaluation/xgboost.csv')

F:\anaconda\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in true_divide
F:\anaconda\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in true_divide


## RandomForest

In [137]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(processed_set_transformed,newdf.Target)
rf.score(processed_set_transformed, newdf.Target)

F:\anaconda\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9455782312925171

In [138]:
eval = printModelEvaluationMetrics(np.array(newdf.Target), np.array(rf.predict(processed_set_transformed)), le.inverse_transform(newdf.Target.unique()))
eval.to_csv(os.getcwd() + '/../evaluation/RandomForest.csv')

## Neural Nets

In [121]:
with open(os.getcwd()+'/../Data/convertcsv.json') as file:
    data = json.load(file)
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

stemmer = LancasterStemmer()
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

In [122]:
tf.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, 16)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 19999  | total loss: 0.02564 | time: 0.091s
| Adam | epoch: 1000 | loss: 0.02564 - acc: 0.9988 -- iter: 152/155
Training Step: 20000  | total loss: 0.02318 | time: 0.094s
| Adam | epoch: 1000 | loss: 0.02318 - acc: 0.9989 -- iter: 155/155
--


In [123]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

results = model.predict([bag_of_words("Hey i am gaurang", words)])[0]
results_index = np.argmax(results)
tag = labels[results_index]
print(tag)
if results[results_index] < 0.7:
    print('I don\'t know')
    print(results[results_index])

greeting


In [124]:
with open(os.getcwd()+"/../models/data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [125]:
model.save(os.getcwd()+'/../models/model.tflearn')

INFO:tensorflow:F:\anaconda\Scripts\dsemt\project\InvestoBot\Training/../models/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [156]:
predictedClasses = np.argmax(model.predict(training), axis=1)
actualClasses = np.argmax(output, axis=1)
eval = printModelEvaluationMetrics(actualClasses, predictedClasses, labels)
eval.to_csv(os.getcwd() + '/../evaluation/neuralnetworks.csv')